### Reference
- http://konlpy.org/ko/latest/
- https://datascienceschool.net/view-notebook/70ce46db4ced4a999c6ec349df0f4eb0/
- https://www.lucypark.kr/slides/2014-pyconkr/
- https://www.lucypark.kr/courses/2015-dm/text-mining.html

In [1]:
import konlpy

# 말뭉치
- http://konlpy.org/ko/latest/api/konlpy.corpus/
- for toy project
- kolaw: 한국 법률 말뭉치
- kobill: 대한민국 국회 의안 말뭉치

In [2]:
from konlpy.corpus import kolaw
kolaw.fileids()

['constitution.txt']

In [3]:
c = kolaw.open('constitution.txt').read()
print(c[:100])

대한민국헌법

유구한 역사와 전통에 빛나는 우리 대한국민은 3·1운동으로 건립된 대한민국임시정부의 법통과 불의에 항거한 4·19민주이념을 계승하고, 조국의 민주개혁과 평화적 통일의


In [4]:
from konlpy.corpus import kobill
kobill.fileids()

['1809890.txt',
 '1809891.txt',
 '1809892.txt',
 '1809893.txt',
 '1809894.txt',
 '1809895.txt',
 '1809896.txt',
 '1809897.txt',
 '1809898.txt',
 '1809899.txt']

In [5]:
d = kobill.open('1809892.txt').read()
print(d[:100])

교육공무원법 일부개정법률안

(정의화의원 대표발의 )

 의 안
 번 호

9892

발의연월일 : 2010.  11.  12.  

발  의  자 : 정의화․이명수․김을동 

이


# 형태소 분석 / 품사 태깅

아래 5개의 형태소 분석 / 품사 태깅 관련 라이브러리를 파이썬으로 포팅한 형태

- Kkma
    - http://kkma.snu.ac.kr/
- Hannanum
    - http://semanticweb.kaist.ac.kr/hannanum/
- Twitter
    - https://github.com/twitter/twitter-korean-text/
- Komoran
    - http://www.shineware.co.kr/?page_id=835
- Mecab
    - https://bitbucket.org/eunjeon/mecab-ko-dic
    
각 라이브러리는 다른 데이터로 학습되었으며, 태깅되는 품사도 다름
- [라이브러리 별 품사 기준](https://docs.google.com/spreadsheets/d/1OGAjUvalBuX-oZvZ_-9tEfYD2gQe7hTGsgUpiiBSXI8/edit#gid=0)


- **가장 빠른 것은 Mecab**
    - 위키피디아/세종코퍼스 등으로만 학습되어 구어체에 약함


- **구어체에 가장 가까운 것은 Twitter**


- 실행시간 비교 http://konlpy.org/ko/latest/morph/

    <img src="http://konlpy.org/ko/latest/_images/time.png">



In [6]:
from konlpy.tag import Hannanum, Kkma, Komoran, Mecab, Twitter 

로딩 시간: 사전 로딩을 포함하여 클래스를 로딩하는 시간.
- Kkma: 5.6988 secs
- Komoran: 5.4866 secs
- Hannanum: 0.6591 secs
- Twitter: 1.4870 secs
- Mecab: 0.0007 secs

실행시간: 10만 문자의 문서를 대상으로 각 클래스의 pos 메소드를 실행하는데 소요되는 시간.
- Kkma: 35.7163 secs
- Komoran: 25.6008 secs
- Hannanum: 8.8251 secs
- Twitter: 2.4714 secs
- Mecab: 0.2838 secs

In [7]:
kkma = Kkma()
komoran = Komoran()
hannanum = Hannanum()
twitter = Twitter()
mecab = Mecab()

In [10]:
sentence = '새 맥북프로 갖고 싶다'

In [11]:
names = ['꼬꼬마', '한나눔', '트위터', '미캡']
engines = [kkma, hannanum, twitter, mecab]

In [12]:
import time
for name, engine in zip(names, engines):
    start = time.time()
    print(f'{name}: {engine.pos(sentence)} | {time.time()-start:.3f}s')
    time.sleep(1)

꼬꼬마: [('새', 'MDT'), ('맥', 'NNG'), ('북', 'NNG'), ('프로', 'NNG'), ('갖', 'VV'), ('고', 'ECE'), ('싶', 'VXA'), ('다', 'EFN')] | 6.507s
한나눔: [('새', 'M'), ('맥북프', 'N'), ('로', 'J'), ('갖', 'P'), ('고', 'E'), ('싶', 'P'), ('다', 'E')] | 0.663s
트위터: [('새', 'Noun'), ('맥북프로', 'Noun'), ('갖고', 'Verb'), ('싶다', 'Verb')] | 2.926s
미캡: [('새', 'MM'), ('맥북', 'NNP'), ('프로', 'NNG'), ('갖', 'VV'), ('고', 'EC'), ('싶', 'VX'), ('다', 'EC')] | 0.017s


### Komoran has issue..
- it should not be used with other taggers in single session

In [11]:
# names = ['코모란']
# engines = [komoran]

In [12]:
# import time
# for name, engine in zip(names, engines):
#     start = time.time()
#     print(f'{name}: {engine.pos(sentence)} | {time.time()-start:.2f}s')
#     time.sleep(1)

In [13]:
mecab.pos(sentence)

[('새', 'MM'),
 ('맥북', 'NNP'),
 ('프로', 'NNG'),
 ('갖', 'VV'),
 ('고', 'EC'),
 ('싶', 'VX'),
 ('다', 'EC')]

In [14]:
twitter.pos(sentence)

[('새', 'Noun'), ('맥북프로', 'Noun'), ('갖고', 'Verb'), ('싶다', 'Verb')]

# 명사 필터링
- 대부분의 경우 조사/어미 등을 날리고 명사/동사만을 feature로 이용

In [15]:
mecab.nouns(sentence)

['맥북', '프로']

In [16]:
twitter.nouns(sentence)

['새', '맥북프로']